In [ ]:
import yaml
import numpy as np
import pandas as pd
import lhapdf as lh
import matplotlib.pyplot as plt
import MatplotlibSettings
from collections import deque
import copy
import os
from os import listdir
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
# Fit folder
FitFolder = "./"

In [ ]:
# Read and report chi2's
with open(FitFolder + "/Chi2s.yaml", "r") as chi2file:
    for chi2 in yaml.load(chi2file, Loader = yaml.CLoader):
        for c in chi2.items(): 
            print(c[0],"(Npt = ",c[1]["Npt"],"): chi2 = ", c[1]["chi2"])

In [ ]:
# Plot settings
exps = [
    "HERMES $\pi^-$ deuteron",
    "HERMES $\pi^-$ proton",
    "HERMES $\pi^+$ deuteron",
    "HERMES $\pi^+$ proton",
    r"COMPASS $\pi^-$",
    r"COMPASS $\pi^+$",
    r"BELLE $\pi^\pm$",
    r"BABAR prompt $\pi^\pm$",
    r"TASSO 12 GeV $\pi^\pm$",
    r"TASSO 14 GeV $\pi^\pm$",
    r"TASSO 22 GeV $\pi^\pm$",
    r"TPC $\pi^\pm$",
    r"TASSO 30 GeV $\pi^\pm$",
    r"TASSO 34 GeV $\pi^\pm$",
    r"TASSO 44 GeV $\pi^\pm$",
    r"TOPAZ $\pi^\pm$",
    r"ALEPH $\pi^\pm$",
    r"DELPHI total $\pi^\pm$",
    r"DELPHI $uds$ $\pi^\pm$",
    r"DELPHI bottom $\pi^\pm$",
    r"OPAL $\pi^\pm$",
    r"SLD total $\pi^\pm$",
    r"SLD $uds$ $\pi^\pm$",
    r"SLD charm $\pi^\pm$",
    r"SLD bottom $\pi^\pm$"
]

plotfile = dict(zip(exps, [
    r"HERMES_pim_deut",
    r"HERMES_pim_prot",
    r"HERMES_pip_deut",
    r"HERMES_pip_prot",
    r"COMPASS_pim",
    r"COMPASS_pip",
    r"BELLE_pi",
    r"BABAR_pi",
    r"TASSO_12_pi",
    r"TASSO_14_pi",
    r"TASSO_22_pi",
    r"TPC_pi",
    r"TASSO_30_pi",
    r"TASSO_34_pi",
    r"TASSO_44_pi",
    r"TOPAZ_pi",
    r"ALEPH_pi",
    r"DELPHI_total_pi",
    r"DELPHI_uds_pi",
    r"DELPHI_bottom_pi",
    r"OPAL_pi",
    r"SLD_total_pi",
    r"SLD_uds_pi",
    r"SLD_charm_pi",
    r"SLD_bottom_pi"
]))

ylabels = dict(zip(exps, [
    r"$\displaystyle\frac{dM^{\pi^-}}{dz}$",
    r"$\displaystyle\frac{dM^{\pi^-}}{dz}$",
    r"$\displaystyle\frac{dM^{\pi^+}}{dz}$",
    r"$\displaystyle\frac{dM^{\pi^+}}{dz}$",
    r"$\displaystyle\frac{dM^{\pi^-}}{dz}$",
    r"$\displaystyle\frac{dM^{\pi^+}}{dz}$",
    r"$\displaystyle\frac{d\sigma}{dz}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dz}$",
    r"$\displaystyle\frac{s}{\beta}\frac{d\sigma}{dz}$",
    r"$\displaystyle\frac{s}{\beta}\frac{d\sigma}{dz}$",
    r"$\displaystyle\frac{s}{\beta}\frac{d\sigma}{dz}$",
    r"$\displaystyle\frac{1}{\beta\sigma}\frac{d\sigma}{dz}$",
    r"$\displaystyle\frac{s}{\beta}\frac{d\sigma}{dz}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dx_p}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dx_p}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{d\xi}$",    
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dx_p}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dp_h}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dp_h}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dp_h}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dp_h}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dx_p}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dx_p}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dx_p}$",
    r"$\displaystyle\frac{1}{\sigma}\frac{d\sigma}{dx_p}$"
]))

binints = dict(zip(exps, [
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    False,
    False,
    False,
    True,
    False,
    False,
    False,
    False,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True
]))

xlog = dict(zip(exps, [
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True
]))

xlims = dict(zip(exps, [
    [0.2, 0.8],
    [0.2, 0.8],
    [0.2, 0.8],
    [0.2, 0.8],
    [0.01, 1],
    [0.01, 1],
    [0.18, 1],
    [0.01, 1],
    [0.05, 0.3],
    [0.04, 0.6],
    [0.01, 0.6],
    [0.01, 1],
    [0.02, 0.1],
    [0.02, 1],
    [0.01, 0.8],
    [0.008, 0.3],
    [0.005, 1],
    [0.01, 1],
    [0.01, 1],
    [0.01, 1],
    [0.005, 1],
    [0.005, 1],
    [0.005, 1],
    [0.005, 1],
    [0.005, 1]
]))

ylims1 = dict(zip(exps, [
    [0, 2],
    [0, 2],
    [0, 2],
    [0, 2],
    [],
    [],
    [5e3, 3e7],
    [2e-3, 10],
    [0.8, 30],
    [0.05, 20],
    [0.02, 15],
    [0.02, 200],
    [0.2, 7],
    [0.05, 200],
    [0.2, 300],
    [0.8, 10],
    [0.03, 1000],
    [0.00005, 10],
    [0.0002, 10],
    [0.00002, 10],
    [0.0002, 20],
    [0.005, 1000],
    [0.005, 1000],
    [0.002, 1000],
    [0.002, 1000],
    [0.002, 1000]
]))

ylims2 = dict(zip(exps, [
    [0.6, 1.4],
    [0.6, 1.4],
    [0.6, 1.4],
    [0.6, 1.4],
    [0.5, 1.5],
    [0.5, 1.5],
    [0.8, 1.2],
    [0.95, 1.05],
    [0.5, 1.5],
    [0.5, 1.5],
    [0.5, 1.5],
    [0.5, 1.5],
    [0.5, 1.5],
    [0.5, 1.5],
    [0.5, 1.5],
    [0.5, 1.5],
    [0.5, 1.5],
    [0.8, 1.2],
    [0.8, 1.2],
    [0.8, 1.2],
    [0.8, 1.2],
    [0.8, 1.2],
    [0.8, 1.2],
    [0.8, 1.2],
    [0.8, 1.2]
]))

# Create plots folder
if not os.path.exists(FitFolder + "/plots"):
    os.makedirs(FitFolder + "/plots")

In [ ]:
# Chi2 profiles
for frep in [f.replace(".yaml", "") for f in sorted(listdir(FitFolder + "/log/"))]:
    with open(FitFolder + "/log/" + frep + ".yaml", "r") as rep:
        replica = yaml.load(rep, Loader = yaml.CLoader)
        ite   = [r["iteration"]       for r in replica]
        chi2t = [r["training chi2"]   for r in replica]
        chi2v = [r["validation chi2"] for r in replica]
        
        plt.title(r"\textbf{" + frep.replace("_", " ") + "}")
        plt.ylabel(r"\textbf{Error function}")
        plt.xlabel(r"\textbf{Iteration}")
        plt.xscale("log")
        plt.yscale("log")
        plt.ylim(1, 10)
        plt.xlim(10, 3000)
        plt.plot(ite, chi2t, color = "blue", label = r"\textbf{Training}")
        plt.plot(ite, chi2v, color = "red",  label = r"\textbf{Validation}")
        plt.legend()
        #plt.savefig(FitFolder + "/" + frep + ".pdf")
        plt.show()
        plt.close()

In [ ]:
# SIA experiments
with open(FitFolder + "/Predictions.yaml", "r") as p:
    preds = yaml.load(p, Loader = yaml.CLoader)
    for e in preds:
        for exp, points in e.items():
            if ("COMPASS" in exp) or ("HERMES" in exp):
                continue
            print(exp)
            zav  = np.array([pt["zav"] for pt in points])
            zmin = np.array([pt["zmin"] for pt in points])
            zmax = np.array([pt["zmax"] for pt in points])
            excv = np.array([pt["exp. central value"] for pt in points])
            exun = np.array([pt["exp. unc."] for pt in points])
            thcv = np.array([pt["prediction"] for pt in points])
            thun = np.array([pt["pred. unc."] for pt in points])

            f, (ax1, ax2) = plt.subplots(2, 1, sharex = "all", gridspec_kw = dict(width_ratios = [1], height_ratios = [4, 1]))
            plt.subplots_adjust(wspace = 0, hspace = 0)

            # Upper panel
            ax1.set_xlim(xlims[exp])
            ax1.set_ylim(ylims1[exp])
            ax1.set_ylabel(ylabels[exp])
            ax1.set_yscale("log")
            if xlog[exp]:
                ax1.set_xscale("log")
            ax1.errorbar(zav, excv, exun, elinewidth = 2, capsize = 3, capthick = 1.5, label = r"\textbf{" + exp + " data}", markersize = 5, fmt = "ko")
            if binints[exp]:
                ax1.bar(zmin, 2 * thun, bottom = thcv - thun, width = zmax - zmin, align = "edge", color = "red", label = r"\textbf{NLO + MAP FF1.0}", alpha = 0.5)
            else:
                ax1.errorbar(zav, thcv, thun, elinewidth = 2, capsize = 3, capthick = 1.5, label = r"\textbf{NLO + MAP FF1.0}", markersize = 4, fmt = "rs")
            ax1.legend(fontsize = 20)

            # Lower panel
            ax2.set_xlim(xlims[exp])
            ax2.set_ylim(ylims2[exp])
            ax2.set_xlabel(r"$z$")
            ax2.set_ylabel(r"\textbf{Ratio to Data}", fontsize = 16)
            if xlog[exp]:
                ax2.set_xscale("log")
            ax2.axhline(y = 1, c = "k", ls = "--", lw = 1.5)
            ax2.errorbar(zav, excv/excv, exun/excv, elinewidth = 2, capsize = 3, capthick = 1.5, markersize = 5, fmt = "ko")
            if binints[exp]:
                ax2.bar(zmin, 2 * thun/excv, bottom = (thcv - thun)/excv, width = zmax - zmin, align = "edge", color = "red", alpha = 0.5)
            else:
                ax2.errorbar(zav, thcv/excv, thun/excv, elinewidth = 2, capsize = 3, capthick = 1.5, markersize = 5, fmt = "rs")

            plt.savefig(FitFolder + "/plots/" + plotfile[exp] + ".pdf")
            plt.show()
            plt.close()

In [ ]:
# COMPASS
Qcut = 2
with open(FitFolder + "/Predictions.yaml", "r") as p:
    preds = yaml.load(p, Loader = yaml.CLoader)
    for e in preds:
        for exp, points in e.items():
            if "COMPASS" in exp:
                print(exp)
                Qav  = np.array([pt["Qav"] for pt in points])
                zav  = np.array([pt["zav"] for pt in points])
                zmin = np.array([pt["zmin"] for pt in points])
                zmax = np.array([pt["zmax"] for pt in points])
                excv = np.array([pt["exp. central value"] for pt in points])
                exun = np.array([pt["exp. unc."] for pt in points])
                thcv = np.array([pt["prediction"] for pt in points])
                thun = np.array([pt["pred. unc."] for pt in points])
                # Determine bin bounds
                df = pd.DataFrame(zip([pt["xmin"] for pt in points], [pt["xmax"] for pt in points], [pt["ymin"] for pt in points], [pt["ymax"] for pt in points]))
                lb = deque([0])
                for i in range(len(df[0])):
                    if i > 0 and (df[0][i] != df[0][i-1] or df[1][i] != df[1][i-1] or df[2][i] != df[2][i-1] or df[3][i] != df[3][i-1]):
                        lb.append(i)
                ub = copy.deepcopy(lb)
                ub.rotate(-1)
                bb = list(zip(lb, ub))
                del bb[-1]
                for b in bb:
                    bstr = r"\textbf{$" + str(df[0][b[0]]) + " < x < " + str(df[1][b[0]]) + ", " + str(df[2][b[0]]) + " < y < " + str(df[3][b[0]]) + "$}"
                    inc = "_in"
                    if (max(Qav[b[0]:b[1]]) < Qcut):
                        bstr += r"\textbf{ (not included)}"
                        inc = "_out"

                    f, (ax1, ax2) = plt.subplots(2, 1, sharex = "all", gridspec_kw = dict(width_ratios = [1], height_ratios = [4, 1]))
                    plt.subplots_adjust(wspace = 0, hspace = 0)

                    # Upper panel
                    ax1.set_title(bstr)
                    ax1.set_xlim([min(zmin[b[0]:b[1]]), max(zmax[b[0]:b[1]])])
                    ax1.set_ylabel(ylabels[exp])
                    ax1.errorbar(zav[b[0]:b[1]], excv[b[0]:b[1]], exun[b[0]:b[1]], elinewidth = 2, capsize = 3, capthick = 1.5, label = r"\textbf{" + exp + " data}", markersize = 5, fmt = "ko")
                    ax1.bar(zmin[b[0]:b[1]], 2 * thun[b[0]:b[1]], bottom = thcv[b[0]:b[1]] - thun[b[0]:b[1]], width = zmax[b[0]:b[1]] - zmin[b[0]:b[1]], align = "edge", color = "red", label = r"\textbf{NLO + MAP FF1.0}", alpha = 0.5)
                    ax1.legend(fontsize = 20)

                    # Lower panel
                    ax1.set_xlim([min(zmin[b[0]:b[1]]), max(zmax[b[0]:b[1]])])
                    ax2.set_ylim([0.8, 1.2])
                    ax2.set_xlabel(r"$z$")
                    ax2.set_ylabel(r"\textbf{Ratio to Data}", fontsize = 16)
                    ax2.axhline(y = 1, c = "k", ls = "--", lw = 1.5)
                    ax2.errorbar(zav[b[0]:b[1]], excv[b[0]:b[1]]/excv[b[0]:b[1]], exun[b[0]:b[1]]/excv[b[0]:b[1]], elinewidth = 2, capsize = 3, capthick = 1.5, markersize = 5, fmt = "ko")
                    ax2.bar(zmin[b[0]:b[1]], 2 * thun[b[0]:b[1]]/excv[b[0]:b[1]], bottom = (thcv[b[0]:b[1]] - thun[b[0]:b[1]])/excv[b[0]:b[1]], width = zmax[b[0]:b[1]] - zmin[b[0]:b[1]], align = "edge", color = "red", alpha = 0.5)

                    plt.savefig(FitFolder + "/plots/" + plotfile[exp] + "_" + str(df[0][b[0]]) + "_x_" + str(df[1][b[0]]) + "_" + str(df[2][b[0]]) + "_y_" + str(df[3][b[0]]) + inc + ".pdf")
                    plt.show()
                    plt.close()


In [ ]:
# HERMES
Qcut = 1.52
with open(FitFolder + "/Predictions.yaml", "r") as p:
    preds = yaml.load(p, Loader = yaml.CLoader)
    for e in preds:
        for exp, points in e.items():
            if "HERMES" in exp:
                print(exp)
                Qav  = np.array([pt["Qav"] for pt in points])
                Qmin = np.array([pt["Qmin"] for pt in points])
                Qmax = np.array([pt["Qmax"] for pt in points])
                zav  = np.array([pt["zav"] for pt in points])
                zmin = np.array([pt["zmin"] for pt in points])
                zmax = np.array([pt["zmax"] for pt in points])
                excv = np.array([pt["exp. central value"] for pt in points])
                exun = np.array([pt["exp. unc."] for pt in points])
                thcv = np.array([pt["prediction"] for pt in points])
                thun = np.array([pt["pred. unc."] for pt in points])
                # Determine bin bounds
                df = pd.DataFrame(zip([pt["xmin"] for pt in points], [pt["xmax"] for pt in points], [pt["Qmin"] for pt in points], [pt["Qmax"] for pt in points]))
                lb = deque([0])
                for i in range(len(df[0])):
                    if i > 0 and (df[0][i] != df[0][i-1] or df[1][i] != df[1][i-1] or df[2][i] != df[2][i-1] or df[3][i] != df[3][i-1]):
                        lb.append(i)
                ub = copy.deepcopy(lb)
                ub.rotate(-1)
                bb = list(zip(lb, ub))
                del bb[-1]
                for b in bb:
                    bstr = r"\textbf{$" + str(df[0][b[0]]) + " < x < " + str(df[1][b[0]]) + ", " + str(round(df[2][b[0]],2)) + " \mbox{ GeV} < Q < " + str(round(df[3][b[0]], 2)) + " \mbox{ GeV}$}"
                    inc = "_in"
                    if (max(Qav[b[0]:b[1]]) < Qcut):
                        bstr += r"\textbf{ (not included)}"
                        inc = "_out"

                    f, (ax1, ax2) = plt.subplots(2, 1, sharex = "all", gridspec_kw = dict(width_ratios = [1], height_ratios = [4, 1]))
                    plt.subplots_adjust(wspace = 0, hspace = 0)

                    # Upper panel
                    ax1.set_title(bstr)
                    ax1.set_xlim(xlims[exp])
                    ax1.set_ylim(ylims1[exp])
                    ax1.set_ylabel(ylabels[exp])
                    ax1.errorbar(zav[b[0]:b[1]], excv[b[0]:b[1]], exun[b[0]:b[1]], elinewidth = 2, capsize = 3, capthick = 1.5, label = r"\textbf{" + exp + " data}", markersize = 5, fmt = "ko")
                    ax1.bar(zmin[b[0]:b[1]], 2 * thun[b[0]:b[1]], bottom = thcv[b[0]:b[1]] - thun[b[0]:b[1]], width = zmax[b[0]:b[1]] - zmin[b[0]:b[1]], align = "edge", color = "red", label = r"\textbf{NLO + MAP FF1.0}", alpha = 0.5)
                    ax1.legend(fontsize = 20)

                    # Lower panel
                    ax2.set_xlim(xlims[exp])
                    ax2.set_ylim(ylims2[exp])
                    ax2.set_xlabel(r"$z$")
                    ax2.set_ylabel(r"\textbf{Ratio to Data}", fontsize = 16)
                    ax2.axhline(y = 1, c = "k", ls = "--", lw = 1.5)
                    ax2.errorbar(zav[b[0]:b[1]], excv[b[0]:b[1]]/excv[b[0]:b[1]], exun[b[0]:b[1]]/excv[b[0]:b[1]], elinewidth = 2, capsize = 3, capthick = 1.5, markersize = 5, fmt = "ko")
                    ax2.bar(zmin[b[0]:b[1]], 2 * thun[b[0]:b[1]]/excv[b[0]:b[1]], bottom = (thcv[b[0]:b[1]] - thun[b[0]:b[1]])/excv[b[0]:b[1]], width = zmax[b[0]:b[1]] - zmin[b[0]:b[1]], align = "edge", color = "red", alpha = 0.5)

                    plt.savefig(FitFolder + "/plots/" + plotfile[exp] + "_" + str(df[0][b[0]]) + "_x_" + str(df[1][b[0]]) + "_" + str(df[2][b[0]]) + "_y_" + str(df[3][b[0]]) + inc + ".pdf")
                    plt.show()
                    plt.close()



In [ ]:
# Upload sets and declare uncertainty type
lh.pathsAppend(str(FitFolder))
#ffs = [lh.mkPDFs("LHAPDFSet"), lh.mkPDFs("NNFF10_PIp_nlo"), lh.mkPDFs("DSS14_NLO_Pip"), lh.mkPDFs("JAM19FF_pion_nlo")]
#unc = ["montecarlo", "montecarlo", "hessian", "montecarlo"]
#colff = ["red", "blue", "green", "orange"]
#nameff = [r"\textbf{MAP FF1.0}", r"\textbf{DEHSS14}", r"\textbf{JAM19}"]
ffs = [lh.mkPDFs("LHAPDFSet"), lh.mkPDFs("DSS14_NLO_Pip"), lh.mkPDFs("JAM20-SIDIS_FF_pion_nlo")]
unc = ["montecarlo", "hessian", "montecarlo"]
colff = ["red", "blue", "orange"]
nameff = [r"\textbf{MAP FF1.0}", r"\textbf{DEHSS14}", r"\textbf{JAM20}"]

# Function that returns the central values
def ComputeCentralValueAndUncertainty(x, Q, comb):
    centralvalue = []
    uncertainty  = []
    for s in zip(ffs, unc):
        if s[1] == "hessian":
            # In the case of a hessian set, use replica 0 as central value
            cv = 0
            f = s[0][0].xfxQ(x, Q)
            for k, v in comb.items():
                cv += v * f[k]
            err = 0
            for im in range(int((len(s[0]) - 1)/2)):
                fp = s[0][2*im+1].xfxQ(x, Q)
                fm = s[0][2*im+2].xfxQ(x, Q)
                t = 0
                for k, v in comb.items():
                    t += v * fp[k]
                    t -= v * fm[k]
                err += t**2
            centralvalue.append(cv)
            uncertainty.append(np.sqrt(err)/2)            
        elif s[1] == "montecarlo":
            # In case of a MC set, compute the average and standard deviation (do not include replica 0)
            cv  = 0
            cv2 = 0
            for irep in range(1, len(s[0])):
                f = s[0][irep].xfxQ(x, Q)
                t = 0
                for k, v in comb.items():
                    t += v * f[k]
                cv  += t / ( len(s[0]) - 1 )
                cv2 += t**2 / ( len(s[0]) - 1 )
            centralvalue.append(cv)
            uncertainty.append(np.sqrt(cv2 - cv**2))
        else:
            sys.exit("Unknown error: ", s[1])
    return centralvalue, uncertainty

In [ ]:
# FF plot settings

# Scale
Q = 5

# Grid in x
xv = np.logspace(-2, -0.0001, 1000)

# Combinations to be plotted
combs = [{21: 1}, {1: 1, -1: 1, 2: 1, -2: 1, 3: 1, -3: 1}, {1: 1, -1: 1}, {2: 1, -2: 1}, {3: 1, -3: 1}, {4: 1, -4: 1}, {5: 1, -5: 1}, {3: 1, -3: -1}, {1: 1, -1: -1}]
labels = [r"$zD_g^{\pi^+}(z,Q)$", r"$zD_{d^++u^++s^+}^{\pi^+}(z,Q)$", r"$zD_{d^+}^{\pi^+}(z,Q)$", r"$zD_{u^+}^{\pi^+}(z,Q)$", r"$zD_{s^+}^{\pi^+}(z,Q)$", r"$zD_{c^+}^{\pi^+}(z,Q)$", r"$zD_{b^+}^{\pi^+}(z,Q)$", r"$zD_{s^-}^{\pi^+}(z,Q)$", r"$zD_{d^-}^{\pi^+}(z,Q)$"]
ylim1 = [[-1.8, 2], [-1.8, 8], [-0.5, 4], [-0.5, 4], [-0.5, 4], [-0.5, 4], [-0.5, 4], [-2, 4], [-0.5, 2]]
pdfname = ["Dg", "Ddusp", "Ddp", "Dup", "Dsp", "Dcp", "Dbp", "Dsm", "Ddm"]

In [ ]:
# Plot single replicas of the current fit for all flavours
labelmap = {-5: "\overline{b}", -4: "\overline{c}", -3: "\overline{s}", -2: "\overline{u}", -1: "\overline{d}", 0: "g", 1: "d", 2: "u", 3: "s", 4: "c", 5: "b"}
for ifl in range(-5,6):
    xf = [ffs[0][0].xfxQ(ifl, 0.1, Q) for x in xv]
    
    plt.title(r"$Q = " + str(Q) + r"$ \textbf{GeV}")
    plt.ylabel(r"$zD_{" + labelmap[ifl] + r"}(z, Q)$")
    plt.xlabel(r"$z$")
    plt.xlim(0.05, 1)
    plt.xscale("log")
    plt.ylim(-2, 5)
    for ff in ffs[0]:
        plt.plot(xv, [ff.xfxQ(ifl, x, Q) for x in xv], color = "green", ls = "-", lw = 0.2)
        plt.plot(xv, xv - xv, color = "black", ls = "--", lw = 1)
    plt.show()
    plt.close()



In [ ]:
# Plot FFs
for cm in zip(combs, labels, ylim1, pdfname):
    fv  = []
    dfv = []
    for x in xv:
        cv, err = ComputeCentralValueAndUncertainty(x, Q, cm[0])
        fv.append(cv)
        dfv.append(err)
    # Transpose results
    fv  = list(map(list, zip(*fv)))
    dfv = list(map(list, zip(*dfv)))

    # Setup plot
    f, (ax1, ax2) = plt.subplots(2, 1, sharex = "all", gridspec_kw = dict(width_ratios = [1], height_ratios = [3, 1]))
    plt.subplots_adjust(wspace = 0, hspace = 0)
    
    ax1.set_title(r"$Q = " + str(Q) + r"$ \textbf{GeV}")
    ax1.set_ylabel(cm[1])
    ax1.set_xlim([0.05, 1])
    ax1.set_xscale("log")
    ax1.set_ylim(cm[2])
    
    ax2.set_xlabel(r"$z$")
    ax2.set_ylabel(r"\textbf{Ratio to " + nameff[0] + "}", fontsize = 16)
    ax2.set_xlim([0.05, 1])
    ax2.set_xscale("log")
    ax2.set_ylim([0, 2])

    for iset in range(len(fv)):
        ax1.plot(xv, fv[iset], color = colff[iset], ls = "-", lw = 1)
        ax1.fill_between(xv, np.array(fv[iset]) + np.array(dfv[iset]), np.array(fv[iset]) - np.array(dfv[iset]), color = colff[iset], alpha = 0.3, label = nameff[iset])
        ax1.plot(xv, xv - xv, color = "black", ls = "--", lw = 1)
        ax2.plot(xv, np.array(fv[iset])/np.array(fv[0]), color = colff[iset], ls = "-", lw = 1)
        ax2.fill_between(xv, (np.array(fv[iset])+np.array(dfv[iset]))/np.array(fv[0]), (np.array(fv[iset])-np.array(dfv[iset]))/np.array(fv[0]), color = colff[iset], alpha = 0.3)
    ax1.legend()

    plt.savefig(FitFolder + "/plots/" + cm[3] + ".pdf")
    plt.show()
    plt.close()